In [43]:
import pandas as pd
import string
from os import listdir


full_path = "/Users/josephdespres/Documents/Python/pythonChallenges/data_cleaning_challenge/first-midterm-despresj/data/"


files = listdir("data")
pd.DataFrame(files)
# incidents
# county-city-poverty-data
poverty = pd.read_csv(full_path + files[1])
cities_pop = pd.read_csv(full_path + files[2], encoding='latin1')
cities_pop = cities_pop.rename(columns={"NAME": "city_og", "STNAME": "state"})
county_cities = pd.read_csv(full_path + files[3], encoding='latin1')
gtd = pd.read_csv(full_path + files[0], encoding='latin1')
gtd = gtd.rename(columns = {'provstate': 'state'})

In [84]:
gtd.columns
cities_pop.columns
variables = pd.DataFrame({'gtd': pd.Series(gtd.columns),
              'city_pop': pd.Series(cities_pop.columns),
              'county_cities': pd.Series(county_cities.columns),
              'poverty': pd.Series(poverty.columns)})


variables[0:6]

,gtd,city_pop,county_cities,poverty
0,iyear,SUMLEV,latitude,State
1,imonth,STATE,longitude,Area_Name
2,iday,COUNTY,city,PCTPOVALL_2015
3,state,PLACE,state,PCTPOV017_2015
4,city,city_og,county,MEDHHINC_2015
5,latitude,state,NaN,NaN


In [98]:
cities_pop['city'] = cities_pop['city_og'].str.replace(r'\b[a-z]+\b', "",regex=True)

In [67]:
def city_finder(df, string):
    return df[df['city'].str.contains(string, case=False)]

In [101]:
pd.set_option("display.max_rows", 101)

In [99]:
city_finder(cities_pop, 'tacoma')

,SUMLEV,STATE,COUNTY,PLACE,city_og,state,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,city
51781,61,38,9,0,Tacoma township,North Dakota,63,62,63,62,Tacoma
76371,162,53,0,70000,Tacoma city,Washington,202736,204461,207660,211277,Tacoma
76651,157,53,53,70000,Tacoma city,Washington,202736,204461,207660,211277,Tacoma


In [60]:
city_finder(gtd, 'tac')

,iyear,imonth,iday,state,city,latitude,longitude,vicinity,summary,doubtterr,...,weapsubtype1,weapsubtype1_txt,nkill,nwound,property,propextent,propextent_txt,propvalue,propcomment,scite1
0,2013,1,17,Washington,Tacoma,47.252877,-122.444291,0,01/17/2013: An explosive device detonated outs...,0,...,17.0,Other Explosive Type,0,0,0,NaN,NaN,NaN,NaN,"""Small bomb explodes outside WA Correction..."
141,2016,11,25,Washington,Tacoma,47.154901,-122.368098,0,11/25/2016: Assailants set fire to Our Savior ...,0,...,18.0,Arson/Fire,0,0,1,3.0,Minor (likely < $1 million),-99.0,A church was damaged in this attack.,"""Tacoma church badly damaged by suspected arso..."


In [158]:
# try making an identifyer
gtd['id'] = gtd['city'] + gtd['state']
gtd['id'] = gtd['id'].astype(str)
cities_pop['id'] = cities_pop['id'].astype(str)
cities_pop['id'] = cities_pop['city'] + cities_pop['state']

In [160]:
to_join = cities_pop[['id', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015', 'POPESTIMATE2016']]
to_join = to_join.drop_duplicates()

In [162]:
maybe = gtd.merge(to_join, how='left')
maybe


,iyear,imonth,iday,state,city,latitude,longitude,vicinity,summary,doubtterr,...,propextent,propextent_txt,propvalue,propcomment,scite1,id,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016
0,2013,1,17,Washington,Tacoma,47.252877,-122.444291,0,01/17/2013: An explosive device detonated outs...,0,...,NaN,NaN,NaN,NaN,"""Small bomb explodes outside WA Correction...",TacomaWashington,NaN,NaN,NaN,NaN
1,2013,2,7,California,Corona,33.875294,-117.566438,0,02/07/2013: An assailant opened fire on two Ri...,1,...,NaN,NaN,NaN,NaN,"""Key events in hunt for former Los Angeles pol...",CoronaCalifornia,NaN,NaN,NaN,NaN
2,2013,2,25,Washington,Seattle,47.593200,-122.300567,0,02/25/2013: Assailants set fire to a construct...,0,...,3.0,Minor (likely < $1 million),60000.0,A single family home and construction equipmen...,"""Environmental anarchists claim responsibility...",SeattleWashington,NaN,NaN,NaN,NaN
3,2013,4,15,Massachusetts,Boston,42.350265,-71.080976,0,04/15/2013: An explosive device consisting of ...,0,...,4.0,Unknown,-99.0,Nearby shops and buildings were damaged in thi...,"""Boston Marathon Terror Attack Fast Facts,"" CN...",BostonMassachusetts,NaN,NaN,NaN,NaN
4,2013,4,15,Massachusetts,Boston,42.350265,-71.080976,0,04/15/2013: An explosive device consisting of ...,0,...,4.0,Unknown,-99.0,Nearby shops and buildings were damaged in thi...,"""Boston Marathon Terror Attack Fast Facts,"" CN...",BostonMassachusetts,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,2016,11,25,Washington,Tacoma,47.154901,-122.368098,0,11/25/2016: Assailants set fire to Our Savior ...,0,...,3.0,Minor (likely < $1 million),-99.0,A church was damaged in this attack.,"""Tacoma church badly damaged by suspected arso...",TacomaWashington,NaN,NaN,NaN,NaN
142,2016,11,28,Ohio,Columbus,40.003290,-83.011421,0,11/28/2016: An assailant rammed his vehicle in...,0,...,NaN,NaN,NaN,NaN,"""Ohio State University attack: Suspect dead an...",ColumbusOhio,NaN,NaN,NaN,NaN
143,2016,12,4,District of Columbia,Washington,38.955943,-77.069984,0,12/04/2016: An assailant opened fire inside Co...,0,...,3.0,Minor (likely < $1 million),5744.0,"A computer system, a ping pong table, and lock...","""The Comet Ping Pong Gunman Answers Our Report...",WashingtonDistrict of Columbia,NaN,NaN,NaN,NaN
144,2016,12,7,California,Redlands,34.053546,-117.181537,0,12/07/2016: Assailants set fire to First Bapti...,0,...,3.0,Minor (likely < $1 million),2000.0,A church was damaged in this attack.,"""Arson investigators probe series of fires at ...",RedlandsCalifornia,NaN,NaN,NaN,NaN


In [157]:
maybe = gtd.join(to_join, how='left', on='id')
maybe


ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

AttributeError: 'DataFrame' object has no attribute 'concat'

In [103]:
cities_pop

,SUMLEV,STATE,COUNTY,PLACE,city_og,state,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,city
0,40,1,0,0,Alabama,Alabama,4829479,4843214,4853875,4863300,Alabama
1,162,1,0,124,Abbeville city,Alabama,2631,2619,2616,2603,Abbeville
2,162,1,0,460,Adamsville city,Alabama,4447,4428,4395,4360,Adamsville
3,162,1,0,484,Addison town,Alabama,748,747,740,738,Addison
4,162,1,0,676,Akron town,Alabama,342,337,337,334,Akron
...,...,...,...,...,...,...,...,...,...,...,...
81580,157,56,43,99990,Balance of Washakie County,Wyoming,2744,2697,2704,2669,Balance Washakie County
81581,50,56,45,0,Weston County,Wyoming,7163,7179,7230,7236,Weston County
81582,157,56,45,56215,Newcastle city,Wyoming,3496,3505,3525,3535,Newcastle
81583,157,56,45,79125,Upton town,Wyoming,1094,1101,1107,1106,Upton


In [104]:
gtd

,iyear,imonth,iday,state,city,latitude,longitude,vicinity,summary,doubtterr,...,weapsubtype1,weapsubtype1_txt,nkill,nwound,property,propextent,propextent_txt,propvalue,propcomment,scite1
0,2013,1,17,Washington,Tacoma,47.252877,-122.444291,0,01/17/2013: An explosive device detonated outs...,0,...,17.0,Other Explosive Type,0,0,0,NaN,NaN,NaN,NaN,"""Small bomb explodes outside WA Correction..."
1,2013,2,7,California,Corona,33.875294,-117.566438,0,02/07/2013: An assailant opened fire on two Ri...,1,...,4.0,Rifle/Shotgun (non-automatic),1,1,0,NaN,NaN,NaN,NaN,"""Key events in hunt for former Los Angeles pol..."
2,2013,2,25,Washington,Seattle,47.593200,-122.300567,0,02/25/2013: Assailants set fire to a construct...,0,...,18.0,Arson/Fire,0,0,1,3.0,Minor (likely < $1 million),60000.0,A single family home and construction equipmen...,"""Environmental anarchists claim responsibility..."
3,2013,4,15,Massachusetts,Boston,42.350265,-71.080976,0,04/15/2013: An explosive device consisting of ...,0,...,12.0,Remote Trigger,1,132,1,4.0,Unknown,-99.0,Nearby shops and buildings were damaged in thi...,"""Boston Marathon Terror Attack Fast Facts,"" CN..."
4,2013,4,15,Massachusetts,Boston,42.350265,-71.080976,0,04/15/2013: An explosive device consisting of ...,0,...,12.0,Remote Trigger,2,132,1,4.0,Unknown,-99.0,Nearby shops and buildings were damaged in thi...,"""Boston Marathon Terror Attack Fast Facts,"" CN..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,2016,11,25,Washington,Tacoma,47.154901,-122.368098,0,11/25/2016: Assailants set fire to Our Savior ...,0,...,18.0,Arson/Fire,0,0,1,3.0,Minor (likely < $1 million),-99.0,A church was damaged in this attack.,"""Tacoma church badly damaged by suspected arso..."
142,2016,11,28,Ohio,Columbus,40.003290,-83.011421,0,11/28/2016: An assailant rammed his vehicle in...,0,...,23.0,Knife or Other Sharp Object,1,11,0,NaN,NaN,NaN,NaN,"""Ohio State University attack: Suspect dead an..."
143,2016,12,4,District of Columbia,Washington,38.955943,-77.069984,0,12/04/2016: An assailant opened fire inside Co...,0,...,2.0,Automatic Weapon,0,0,1,3.0,Minor (likely < $1 million),5744.0,"A computer system, a ping pong table, and lock...","""The Comet Ping Pong Gunman Answers Our Report..."
144,2016,12,7,California,Redlands,34.053546,-117.181537,0,12/07/2016: Assailants set fire to First Bapti...,0,...,18.0,Arson/Fire,0,0,1,3.0,Minor (likely < $1 million),2000.0,A church was damaged in this attack.,"""Arson investigators probe series of fires at ..."


In [94]:
cities = gtd['city'].unique()

# gameplan
here we have 3 datasets one on poverty by county, one on population by city, one that will bridge the two. 
Remember cities and counties are written differently (eg st. louis could be saint louis). The goal here will be no missing data.

- make df: `popcounty` by adding `counties` to population-by-city `cities_pop`.
- make df: `povcity` by joining poverty-by-county `cities_pop` and adding cities `county_cities` 
- make df: stats_city_county by combining `povcity` and `popcounty`
- make df: joining this to incident.

## city and population
match by city and state

In [3]:
exec(open("/Users/josephdespres/Documents/Python/pythonChallenges/pandas/usstates.py").read())
# get state abbvs
inv_map = {us_state_abbrev[k] : k for k in us_state_abbrev}
# invert dictionary

In [89]:
county_cities = county_cities.replace({'state': inv_map})
county_cities

city_state_counties = county_cities['city'] + county_cities['state']

NameError: name 'inv_map' is not defined

In [10]:
cities_pop

,SUMLEV,STATE,COUNTY,PLACE,city,state,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016
0,40,1,0,0,Alabama,Alabama,4829479,4843214,4853875,4863300
1,162,1,0,124,Abbeville city,Alabama,2631,2619,2616,2603
2,162,1,0,460,Adamsville city,Alabama,4447,4428,4395,4360
3,162,1,0,484,Addison town,Alabama,748,747,740,738
4,162,1,0,676,Akron town,Alabama,342,337,337,334
...,...,...,...,...,...,...,...,...,...,...
81580,157,56,43,99990,Balance of Washakie County,Wyoming,2744,2697,2704,2669
81581,50,56,45,0,Weston County,Wyoming,7163,7179,7230,7236
81582,157,56,45,56215,Newcastle city,Wyoming,3496,3505,3525,3535
81583,157,56,45,79125,Upton town,Wyoming,1094,1101,1107,1106


In [8]:
cities_pop[['NAME', 'STNAME']]

citystate = cities_pop['NAME'] + cities_pop['STNAME']
citystate


0                           AlabamaAlabama
1                    Abbeville cityAlabama
2                   Adamsville cityAlabama
3                      Addison townAlabama
4                        Akron townAlabama
                       ...                
81580    Balance of Washakie CountyWyoming
81581                 Weston CountyWyoming
81582                Newcastle cityWyoming
81583                    Upton townWyoming
81584      Balance of Weston CountyWyoming
Length: 81585, dtype: object

In [9]:


cities_pop

,SUMLEV,STATE,COUNTY,PLACE,city,state,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016
0,40,1,0,0,Alabama,Alabama,4829479,4843214,4853875,4863300
1,162,1,0,124,Abbeville city,Alabama,2631,2619,2616,2603
2,162,1,0,460,Adamsville city,Alabama,4447,4428,4395,4360
3,162,1,0,484,Addison town,Alabama,748,747,740,738
4,162,1,0,676,Akron town,Alabama,342,337,337,334
...,...,...,...,...,...,...,...,...,...,...
81580,157,56,43,99990,Balance of Washakie County,Wyoming,2744,2697,2704,2669
81581,50,56,45,0,Weston County,Wyoming,7163,7179,7230,7236
81582,157,56,45,56215,Newcastle city,Wyoming,3496,3505,3525,3535
81583,157,56,45,79125,Upton town,Wyoming,1094,1101,1107,1106


In [88]:
gtd[~gtd.provstate.isin(cities_pop['city'])]


AttributeError: 'DataFrame' object has no attribute 'provstate'

In [ ]:
gtd[gtd.provstate.isin(cities_pop['city'])]


In [28]:

county_cities[county_cities['city'].str.contains('City')]

,latitude,longitude,city,state,county
390,42.109748,-72.079455,Charlton City,Massachusetts,Worcester
2274,41.599562,-71.982619,Jewett City,Connecticut,New London
2607,40.758951,-74.056335,Union City,New Jersey,Hudson
2616,40.732760,-74.075485,Jersey City,New Jersey,Hudson
2652,40.726001,-74.047304,Jersey City,New Jersey,Hudson
...,...,...,...,...,...
42345,47.737206,-119.227032,Coulee City,Washington,Grant
42352,47.913065,-119.042562,Electric City,Washington,Grant
42353,48.015361,-118.954893,Elmer City,Washington,Okanogan
42434,46.242040,-119.483517,Benton City,Washington,Benton


In [30]:
cities_pop[cities_pop['city'].str.contains('City|town')]

,SUMLEV,STATE,COUNTY,PLACE,city,state,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016
3,162,1,0,484,Addison town,Alabama,748,747,740,738
4,162,1,0,676,Akron town,Alabama,342,337,337,334
7,162,1,0,1132,Alexander City city,Alabama,14928,14905,14792,14773
9,162,1,0,1396,Allgood town,Alabama,628,627,628,627
10,162,1,0,1660,Altoona town,Alabama,930,927,922,919
...,...,...,...,...,...,...,...,...,...,...
81572,157,56,41,5625,Bear River town,Wyoming,516,521,518,517
81574,157,56,41,48675,Lyman town,Wyoming,2088,2074,2073,2072
81575,157,56,41,55345,Mountain View town,Wyoming,1315,1302,1294,1287
81578,157,56,43,75790,Ten Sleep town,Wyoming,257,253,254,250


In [25]:
cities_and_counties = pd.merge(cities_pop, county_cities,  on=['city', 'state'], how="right")
cities_and_counties

,SUMLEV,STATE,COUNTY,PLACE,city,state,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,latitude,longitude,county
0,162.0,32.0,0.0,9700.0,Carson City,Nevada,53989.0,54385.0,54405.0,54742.0,39.154485,-119.730740,Carson City
1,50.0,32.0,510.0,0.0,Carson City,Nevada,53989.0,54385.0,54405.0,54742.0,39.154485,-119.730740,Carson City
2,157.0,32.0,510.0,9700.0,Carson City,Nevada,53989.0,54385.0,54405.0,54742.0,39.154485,-119.730740,Carson City
3,162.0,32.0,0.0,9700.0,Carson City,Nevada,53989.0,54385.0,54405.0,54742.0,39.135503,-119.758750,Carson City
4,50.0,32.0,510.0,0.0,Carson City,Nevada,53989.0,54385.0,54405.0,54742.0,39.135503,-119.758750,Carson City
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42771,NaN,NaN,NaN,NaN,Klawock,Alaska,NaN,NaN,NaN,NaN,55.554393,-133.090680,Prince Wales Ketchikan
42772,NaN,NaN,NaN,NaN,Metlakatla,Alaska,NaN,NaN,NaN,NaN,55.094325,-131.566827,Prince Wales Ketchikan
42773,NaN,NaN,NaN,NaN,Point Baker,Alaska,NaN,NaN,NaN,NaN,55.517921,-132.003244,Prince Wales Ketchikan
42774,NaN,NaN,NaN,NaN,Ward Cove,Alaska,NaN,NaN,NaN,NaN,55.395359,-131.675370,Ketchikan Gateway


In [26]:

cities_and_counties

missing = cities_and_counties[cities_and_counties['SUMLEV'].isna()]

,SUMLEV,STATE,COUNTY,PLACE,city,state,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,latitude,longitude,county
212,NaN,NaN,NaN,NaN,Holtsville,New York,NaN,NaN,NaN,NaN,40.922326,-72.637078,Suffolk
213,NaN,NaN,NaN,NaN,Holtsville,New York,NaN,NaN,NaN,NaN,40.922326,-72.637078,Suffolk
214,NaN,NaN,NaN,NaN,Holtsville,New York,NaN,NaN,NaN,NaN,40.798994,-73.048985,Suffolk
215,NaN,NaN,NaN,NaN,Adjuntas,Puerto Rico,NaN,NaN,NaN,NaN,18.165273,-66.722583,Adjuntas
216,NaN,NaN,NaN,NaN,Aguada,Puerto Rico,NaN,NaN,NaN,NaN,18.393103,-67.180953,Aguada
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42771,NaN,NaN,NaN,NaN,Klawock,Alaska,NaN,NaN,NaN,NaN,55.554393,-133.090680,Prince Wales Ketchikan
42772,NaN,NaN,NaN,NaN,Metlakatla,Alaska,NaN,NaN,NaN,NaN,55.094325,-131.566827,Prince Wales Ketchikan
42773,NaN,NaN,NaN,NaN,Point Baker,Alaska,NaN,NaN,NaN,NaN,55.517921,-132.003244,Prince Wales Ketchikan
42774,NaN,NaN,NaN,NaN,Ward Cove,Alaska,NaN,NaN,NaN,NaN,55.395359,-131.675370,Ketchikan Gateway
